In [ ]:
from scripts.load_data import load_triplet_news_txt

df_news = load_triplet_news_txt("data/TSLA.txt")
df_news

,title,date_raw,source_code,body
0,Sector Update: Consumer Stocks Slide Late Afte...,Dec. 08,MT,Sector Update: Consumer Stocks Slide Late Afte...
1,Sector Update: Consumer Stocks Fall Monday Aft...,Dec. 08,MT,Sector Update: Consumer Stocks Fall Monday Aft...
2,Tesla to See 'Choppy' Trading Backdrop Over Ne...,Dec. 08,MT,Tesla to See 'Choppy' Trading Backdrop Over Ne...
3,"Tesla Faces Challenging EV Outlook, Morgan Sta...",Dec. 08,MT,"Tesla Faces Challenging EV Outlook, Morgan Sta..."
4,"Analyst recommendations: 3M Company, Exxon Mob...",Dec. 08,Zonebourse,"Analyst recommendations: 3M Company, Exxon Mob..."
...,...,...,...,...
389,TESLA : RBC reaffirms its Buy rating,Jan. 02,ZD,TESLA : RBC reaffirms its Buy rating
390,TESLA : UBS remains a Sell rating,Jan. 02,ZD,TESLA : UBS remains a Sell rating
391,TESLA : Jefferies reiterates its Neutral rating,Jan. 02,ZD,TESLA : Jefferies reiterates its Neutral rating
392,Truist Securities Cuts Price Target on Tesla t...,Jan. 02,MT,Truist Securities Cuts Price Target on Tesla t...


In [ ]:
# drop rows congruent with mod2 and mod3 
df_news = df_news.drop(df_news.index[[i for i in range(len(df_news)) if i % 2 == 0 or i % 3 == 0]])
df_news

In [38]:
# Cell 2 – Sentiment via Ollama + Qwen (headline-level)

import json
from typing import Any, Dict, Optional

try:
    import ollama  # pip install ollama
except ImportError:
    ollama = None
    print("Warning: 'ollama' is not installed. Run `pip install ollama` to use the sentiment functions.")


# Default model; override via env OLLAMA_MODEL if you like
import os
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "gemma3:1b")
ticker = "TSLA"
name = "Tesla Inc."


SENTIMENT_SYSTEM_PROMPT = """\
You are a financial news sentiment analyst focusing on a single stock.

Given a short headline or short blurb about {name}. (ticker: {ticker}),
classify the sentiment with respect to {name}'s FUTURE stock performance as:

- "bullish": mainly positive impact on {ticker}
- "bearish": mainly negative impact on {ticker}
- "neutral": mixed or unclear impact, or {name} only mentioned tangentially

Return a compact JSON object with keys:
- "sentiment": one of ["bullish", "bearish", "neutral"]
- "score": a number between -1 and 1 (bearish=-1, bullish=1, neutral≈0)
- "confidence": number between 0 and 1
- "rationale": short explanation in English (1–3 sentences)

Respond with JSON only (no markdown, no backticks).
"""


def classify_headline_sentiment_ollama(
    text: str,
    *,
    model: str = OLLAMA_MODEL,
    max_chars: int = 800,
) -> Dict[str, Any]:
    """
    Classify sentiment of a short headline/blurb using a local Ollama model.

    Parameters
    ----------
    text : str
        Headline or very short summary.
    model : str
        Ollama model name (e.g. 'qwen3:4b').
    max_chars : int
        Max characters to send (just to be safe).

    Returns
    -------
    dict with keys:
        sentiment, score, confidence, rationale, raw
    where 'raw' is the raw model output (for debugging).
    """
    if ollama is None:
        raise RuntimeError("Ollama client not available. Install with `pip install ollama`.")

    snippet = (text or "").strip()
    if len(snippet) > max_chars:
        snippet = snippet[:max_chars] + "... [truncated]"

    user_payload = {
        "headline": snippet,
        "ticker": ticker,
    }

    user_text = (
        "Analyze the following headline about {name} ({ticker}).\n\n"
        + json.dumps(user_payload, ensure_ascii=False, indent=2)
        + "\n\nReturn only the JSON object as specified."
    )

    response = ollama.chat(
        model=model,
        messages=[
            {"role": "system", "content": SENTIMENT_SYSTEM_PROMPT},
            {"role": "user", "content": user_text},
        ],
    )
    raw = response["message"]["content"].strip()

    out: Dict[str, Any] = {
        "sentiment": None,
        "score": None,
        "confidence": None,
        "rationale": None,
        "raw": raw,
    }

    # Try robust JSON extraction: find first '{' and last '}'.
    try:
        start = raw.find("{")
        end = raw.rfind("}")
        if start != -1 and end != -1 and end > start:
            obj = json.loads(raw[start : end + 1])
        else:
            obj = json.loads(raw)

        out["sentiment"] = obj.get("sentiment")
        out["score"] = obj.get("score")
        out["confidence"] = obj.get("confidence")
        out["rationale"] = obj.get("rationale")
    except Exception as e:
        print(f"[classify_headline_sentiment_ollama] JSON parse error: {e}")

    return out


def score_news_df_with_ollama(
    df: pd.DataFrame,
    *,
    text_col: str = "body",
    model: str = OLLAMA_MODEL,
    limit: Optional[int] = None,
) -> pd.DataFrame:
    """
    Apply headline sentiment classification to a DataFrame.

    Parameters
    ----------
    df : DataFrame
        Must contain column `text_col` (e.g. 'body' or 'title').
    text_col : str
        Column used as input text to the model (we'll use 'body' by default).
    model : str
        Ollama model name.
    limit : Optional[int]
        If set, only the first `limit` rows are scored (for quick testing).

    Returns
    -------
    DataFrame : original df with additional columns:
        ['sentiment', 'sentiment_score', 'sentiment_confidence',
         'sentiment_rationale', 'sentiment_raw']
    """
    scored = df.copy()
    n = len(scored)
    if limit is not None:
        n = min(n, limit)

    sentiments: List[Dict[str, Any]] = []
    for i in range(n):
        text = str(scored.iloc[i][text_col])
        print(f"[score_news_df_with_ollama] {i+1}/{n}: {text[:90]}...")
        res = classify_headline_sentiment_ollama(text, model=model)
        sentiments.append(res)

    # For any remaining rows (if limit < len(df)), fill with None
    for _ in range(len(scored) - n):
        sentiments.append(
            {
                "sentiment": None,
                "score": None,
                "confidence": None,
                "rationale": None,
                "raw": None,
            }
        )

    scored["sentiment"] = [s.get("sentiment") for s in sentiments]
    scored["sentiment_score"] = [s.get("score") for s in sentiments]
    scored["sentiment_confidence"] = [s.get("confidence") for s in sentiments]
    scored["sentiment_rationale"] = [s.get("rationale") for s in sentiments]
    scored["sentiment_raw"] = [s.get("raw") for s in sentiments]

    return scored

#keep the first 5 rows for testing
df_scored_rag = score_news_df_with_ollama(df_news, limit=None)
df_scored_rag.to_csv("data/TSLA_sentiment.csv", index=False)


[score_news_df_with_ollama] 1/131: Sector Update: Consumer Stocks Fall Monday Afternoon...
[score_news_df_with_ollama] 2/131: Morgan Stanley Downgrades Tesla to Equalweight From Overweight, Lifts Price Target to $425...
[score_news_df_with_ollama] 3/131: BNP Paribas Raises Tesla Price Target to $313 From $307...
[score_news_df_with_ollama] 4/131: Nvidia Sell-Off a 'DeepSeek Moment,' AI Revolution Just Beginning, Wedbush Says...
[score_news_df_with_ollama] 5/131: Tesla AI Projects Face Uncertainty Despite Retention of Musk as CEO, Truist Says...
[score_news_df_with_ollama] 6/131: Tesla Needs to Keep Elon Musk as CEO, Ives Says...
[score_news_df_with_ollama] 7/131: TESLA : UBS remains a Sell rating...
[score_news_df_with_ollama] 8/131: Tesla to See 'Overwhelming' Shareholder Support for Musk's Pay Package, xAI Investment, We...
[score_news_df_with_ollama] 9/131: Deutsche Bank Adjusts Price Target on Tesla to $470 From $440, Maintains Buy Rating...
[score_news_df_with_ollama] 10/131: Pres

In [39]:
df_scored_rag

,title,date_raw,source_code,body,sentiment,sentiment_score,sentiment_confidence,sentiment_rationale,sentiment_raw
1,Sector Update: Consumer Stocks Fall Monday Aft...,Dec. 08,MT,Sector Update: Consumer Stocks Fall Monday Aft...,bearish,-0.6,0.8,The headline indicates a decline in consumer s...,"```json\n{\n ""sentiment"": ""bearish"",\n ""scor..."
5,Morgan Stanley Downgrades Tesla to Equalweight...,Dec. 08,MT,Morgan Stanley Downgrades Tesla to Equalweight...,bearish,-0.8,0.7,Downgrades from overweight to equalweight sugg...,"```json\n{\n ""sentiment"": ""bearish"",\n ""scor..."
7,BNP Paribas Raises Tesla Price Target to $313 ...,Dec. 01,MT,BNP Paribas Raises Tesla Price Target to $313 ...,bullish,0.8,0.9,The headline indicates a price increase for Te...,"```json\n{\n ""sentiment"": ""bullish"",\n ""scor..."
11,"Nvidia Sell-Off a 'DeepSeek Moment,' AI Revolu...",Nov. 21,MT,"Nvidia Sell-Off a 'DeepSeek Moment,' AI Revolu...",bearish,-0.8,0.7,"The headline explicitly mentions a sell-off, i...","```json\n{\n ""sentiment"": ""bearish"",\n ""scor..."
13,Tesla AI Projects Face Uncertainty Despite Ret...,Nov. 10,MT,Tesla AI Projects Face Uncertainty Despite Ret...,bearish,-0.6,0.8,The headline highlights potential challenges t...,"```json\n{\n ""sentiment"": ""bearish"",\n ""scor..."
...,...,...,...,...,...,...,...,...,...
379,BofA Downgrades Tesla to Neutral From Buy,Jan. 07,MT,BofA Downgrades Tesla to Neutral From Buy,bearish,-0.8,0.7,The downgrade from buy by Bank of America sugg...,"```json\n{\n ""sentiment"": ""bearish"",\n ""scor..."
383,Tesla: on the rebound with broker's comments,Jan. 03,Zonebourse,Tesla: on the rebound with broker's comments,bullish,0.7,0.8,The headline explicitly mentions 'broker's com...,"```json\n{\n ""sentiment"": ""bullish"",\n ""scor..."
385,TESLA : price target raised at Canaccord,Jan. 03,Zonebourse,TESLA : price target raised at Canaccord,bearish,-0.8,0.7,The headline indicates a raised price target s...,"```json\n{\n ""sentiment"": ""bearish"",\n ""scor..."
389,TESLA : RBC reaffirms its Buy rating,Jan. 02,ZD,TESLA : RBC reaffirms its Buy rating,bullish,0.7,0.9,The phrase 'reaffirms its Buy rating' suggests...,"```json\n{\n ""sentiment"": ""bullish"",\n ""scor..."
